<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1_cross_validation_Federica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import csv

import numpy as np
from numpy import savetxt

import pandas as pd
from pandas import DataFrame
from pandas import read_csv
from pandas import concat

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence

from sklearn.utils import compute_class_weight
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
from matplotlib.pyplot import legend

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
def generate_model_leaky(shape):
  
  model = Sequential()
  
  model.add(BatchNormalization())
  model.add(Dense(57, input_dim=shape, kernel_initializer='random_uniform',  bias_initializer='zeros'))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dropout(0.1))
  
  model.add(BatchNormalization())
  model.add(Dense(113, kernel_initializer='random_uniform',  bias_initializer='zeros'))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dropout(0.1))
  
  model.add(BatchNormalization())
  model.add(Dense(226, kernel_initializer='random_uniform',  bias_initializer='zeros'))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dropout(0.1))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [23]:
#LOAD DATA
print("Loading data...")

dataset = '/root/data/6_uHoo_featureDataset.arff'

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("Converting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

Loading data...
DATASET LOADED
Converting values...
CONVERSION DONE


In [24]:
#OPTIMIZERS

#sgd = optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
#rms = optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0) #It is recommended to leave the parameters of this optimizer at their default values (except the learning rate, which can be freely tuned).
#adg = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0) #It is recommended to leave the parameters of this optimizer at their default values.
#ada = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0) #It is recommended to leave the parameters of this optimizer at their default values.
#ama = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
#nad = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004) #It is recommended to leave the parameters of this optimizer at their default values.
adm = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#LOSS

#loss = 'mean_squared_error'
#loss = 'mean_squared_logarithmic_error'
loss = 'binary_crossentropy'

#DEFINE 10-FOLD CROSS-VALIDATION
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
predictions = []
actual = []

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  model = generate_model_leaky(X[train].shape[-1])

  #COMPILE MODEL
  model.compile(loss = loss, optimizer = adm , metrics=['accuracy'])
  
  #EARLY STOPPING
  es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

  #FIT MODEL
  history = model.fit(X[train], Y[train], epochs=10, validation_split=0.33, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight, callbacks = [es])

  #EVALUATE MODEL
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 0)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS AND SAVE IN A CSV FILE
  pred = model.predict_classes(X[test], batch_size=128, verbose=0)
  predictions.append([i,pred])
  actual.append([i,Y[test]])
  #print(predictions[0])
  #print(actual[0])
  
  i+=1

print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



FOLD: 1
Train on 138647 samples, validate on 68289 samples
Epoch 1/10
138647/138647 [==============================] - 14s 104us/step - loss: 0.5210 - acc: 0.7336 - val_loss: 0.5767 - val_acc: 0.7748
Epoch 2/10
138647/138647 [==============================] - 10s 70us/step - loss: 0.4638 - acc: 0.7824 - val_loss: 0.5590 - val_acc: 0.7613
Epoch 3/10
138647/138647 [==============================] - 10s 69us/step - loss: 0.4377 - acc: 0.7888 - val_loss: 0.5783 - val_acc: 0.7568
Epoch 4/10
138647/138647 [==============================] - 10s 69us/step - loss: 0.4231 - acc: 0.7956 - val_loss: 0.5957 - val_acc: 0.7248
Test loss: 45.36%
Test accuracy: 78.76%

FOLD: 2
Train on 138647 samples, validate on 68289 samples
Epoch 1/10
138647/138647 [==============================] - 15s 106us/step - loss: 0.4882 - acc: 0.7764 - val_loss: 0.5861 - val_acc: 0.7083
Epoch 2/10
138647/138647 [==============================] - 10s 70us/step - loss: 0.4445 - acc: 0.7858 - val_loss: 0.5981 - val_acc: 0.697

Save model:

model.save('my_model1_cv.h5')

Load a saved medel:

model = load_model('my_model1_cv.h5')